In [1]:
import requests
from bs4 import BeautifulSoup
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [2]:
CLIENT_ID = "Write your Client ID of spotify here"
CLIENT_SECRET = "Write your Client Secret code of spotify here"

In [ ]:
# Scraping Billboard 100
date = input("Which era of music would you like to explore? Enter the date in the format YYYY-MM-DD to uncover the top songs from that time! ")
URL = f"https://www.billboard.com/charts/hot-100/{date}/"
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
songs = soup.find_all(name="h3", id="title-of-a-story", class_="c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 lrv-u-font-size-18@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-330 u-max-width-230@tablet-only")
song_names = [song.getText().strip("\n\t") for song in songs]

In [ ]:
# Spotify Authentication
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri="http://localhost:8888/callback",  # Changed port, You can add 5000, 5001, 5002 ports too.
    scope="playlist-modify-private user-read-private",
    show_dialog=True,
    cache_path="token.txt"
))
user_id = sp.current_user()["id"]

In [ ]:
# Searching Spotify for songs by title
song_uris = []
year = date.split("-")[0]
for song in song_names:
    result = sp.search(q=f"track:{song} year:{year}", type="track")
    try:
        uri = result["tracks"]["items"][0]["uri"]
        song_uris.append(uri)
    except IndexError:
        print(f"{song} doesn't exist on Spotify. Skipped.")

In [ ]:
# Creating a new private playlist in Spotify
playlist = sp.user_playlist_create(user=user_id, name=f"{date} Billboard 100", public=False)
sp.playlist_add_items(playlist_id=playlist["id"], items=song_uris)

In [ ]:
# Extracting audio features for the songs in the playlist
audio_features = []
for song_uri in song_uris:
    features = sp.audio_features(song_uri)[0]
    audio_features.append(features)

In [ ]:
# Convert audio features to DataFrame
features_df = pd.DataFrame(audio_features)
features_df = features_df[['danceability', 'energy', 'tempo', 'valence']]

In [ ]:
features_df

In [ ]:
def get_recommendations(features_df, n_neighbors=5):
    # Fit KNN to the playlist songs' audio features
    knn = NearestNeighbors(n_neighbors=n_neighbors)

    # Convert DataFrame to numpy array (samples, features)
    knn.fit(features_df.values)

    # Use KNN to recommend similar songs (comparing each song with others)
    distances, indices = knn.kneighbors(features_df.values)

    # Get song URIs based on KNN recommendations (excluding self-references)
    recommended_song_uris = []
    for i in range(len(indices)):
        for idx in indices[i]:
            if idx != i:  # Exclude the song itself
                recommended_song_uris.append(song_uris[idx])

    return list(set(recommended_song_uris))  # Remove duplicates

In [ ]:
# Get recommended songs based on audio features
recommended_song_uris = get_recommendations(features_df)

In [ ]:
# Add the recommended songs to the playlist
if recommended_song_uris:
    sp.playlist_add_items(playlist_id=playlist["id"], items=recommended_song_uris)

print(f"Added {len(recommended_song_uris)} recommended songs to the playlist.")